In [60]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
#pip install category_encoders
import category_encoders as ce

from sklearn.model_selection import train_test_split , GridSearchCV , cross_val_score , RandomizedSearchCV,KFold,RepeatedKFold

from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder , LabelEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import get_scorer
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import learning_curve
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_percentage_error
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import shap
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.feature_selection  import RFE
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PowerTransformer


from matplotlib.colors import ListedColormap
from sklearn.model_selection import cross_val_score
from sklearn import svm 
from sklearn.svm import LinearSVC
from itertools import product
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.inspection import permutation_importance
from IPython.display import display
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report, RocCurveDisplay, DetCurveDisplay,PrecisionRecallDisplay
from sklearn.metrics import roc_auc_score
import math

In [16]:
# обработка тестового фрейма, чтобы потом нужные команды перенести в финальный файл
df1 = pd.read_csv('test.csv')
# так как я убирал в БЛОКЕ 2, то тут тоже не понадобится эта колонка
df1 = df1.drop(columns=['ID'])
df1.head()

,Brand,Model,Year_of_production,Type,Leather_interior,Fuel_type,Engine_volume,Mileage,Cylinders,Gear_box,Drive_wheels,Doors,Color,Airbags,Tax
0,TOYOTA,Prius,2009,Hatchback,No,Hybrid,1.8,174000 km,4.0,Variator,Front,04-May,Blue,8,878
1,HONDA,Civic,2014,Coupe,Yes,Petrol,2.4,58000 km,4.0,Automatic,Front,04-May,Black,4,749
2,SUBARU,Forester,2015,Jeep,Yes,Petrol,2.5,90392 km,4.0,Automatic,4x4,04-May,White,0,934
3,HYUNDAI,Sonata,2018,Sedan,Yes,Petrol,2,41895 km,4.0,Automatic,Front,04-May,White,4,1079
4,JEEP,Compass,2009,Jeep,Yes,Petrol,2.4,163866 km,4.0,Automatic,Front,04-May,Red,4,1141


In [17]:
df1.describe()

,Year_of_production,Cylinders,Airbags,Tax
count,2297.000000,2297.000000,2297.000000,2297.000000
mean,2013.277754,4.366565,6.276448,843.168045
std,2.834180,0.995056,3.927337,337.645818
min,1995.000000,1.000000,0.000000,87.000000
25%,2011.000000,4.000000,4.000000,640.000000
50%,2013.000000,4.000000,4.000000,777.000000
75%,2015.000000,4.000000,10.000000,971.000000
max,2020.000000,16.000000,16.000000,5603.000000


In [18]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2297 entries, 0 to 2296
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Brand               2297 non-null   object 
 1   Model               2297 non-null   object 
 2   Year_of_production  2297 non-null   int64  
 3   Type                2297 non-null   object 
 4   Leather_interior    2297 non-null   object 
 5   Fuel_type           2297 non-null   object 
 6   Engine_volume       2297 non-null   object 
 7   Mileage             2297 non-null   object 
 8   Cylinders           2297 non-null   float64
 9   Gear_box            2297 non-null   object 
 10  Drive_wheels        2297 non-null   object 
 11  Doors               2297 non-null   object 
 12  Color               2297 non-null   object 
 13  Airbags             2297 non-null   int64  
 14  Tax                 2297 non-null   int64  
dtypes: float64(1), int64(3), object(11)
memory usage: 269.3

### Посмотрю схожие несоотвествия данных, как и в train

In [19]:
for i in df1.columns:
    print(i,df1[i].unique())
    
#обьем двигателя = 0  и 20
# интересно, как и в траин

Brand ['TOYOTA' 'HONDA' 'SUBARU' 'HYUNDAI' 'JEEP' 'BMW' 'FORD' 'NISSAN' 'LEXUS'
 'MITSUBISHI' 'MERCEDES-BENZ' 'KIA' 'CHEVROLET' 'SSANGYONG' 'RENAULT'
 'OPEL' 'DAEWOO' 'AUDI' 'VOLKSWAGEN' 'DODGE' 'MAZDA' 'PORSCHE' 'MINI'
 'JAGUAR' 'LAND ROVER' 'FIAT' 'BUICK' 'SUZUKI' 'ACURA' 'CHRYSLER']
Model ['Prius' 'Civic' 'Forester' 'Sonata' 'Compass' '535' 'Fusion' 'Genesis'
 'RAV 4' 'X-Terra' 'GX 470' 'Camry' 'IS 350' 'H1' 'Santa FE'
 'Prius C ჰიბრიდი' 'Outlander' 'Elantra' 'Insight EX' 'S 550' 'Juke'
 'Tucson' 'Optima' 'Fiesta' 'HS 250h' 'Orlando' 'Volt' 'Transit Connect'
 'Fusion SE' 'CHR' 'Aqua' 'Cruze' 'Sonata hybrid' 'Camry SE' 'GLA 250'
 'Tacoma' 'REXTON' 'GLE 350' 'Actyon' 'Megane' '528' 'ML 350' 'GX 460'
 'Astra' 'E 350' 'Captiva' 'Lacetti' 'Highlander' 'Impala' 'Q5' 'Insight'
 'X5' 'Prius C' '740' 'Jetta SE' 'I30' 'Traverse' 'RX 450' '328' 'Vitz'
 'Veloster' 'E 220' 'Focus' 'FIT' 'Challenger' 'Mazda 6' 'Combo' 'Cayman'
 'Cruze LT' 'Malibu LT' 'Malibu' '1300' 'Tundra' 'Countryman' 'Mazda 3

In [20]:
# Функции для обработки колонок с буквами, а также для их разделения
def volume(s):
    if len(s) <= 3:
        a = s
    else:
        a = s[:3]
    return a;

def turbo(df):
    b = []
    for i in range(len(df)):
        if df['Engine_volume'][i][4:] == 'Turbo':
            b.append(1)    
        else:
            b.append(0)
    return pd.Series(b);
# Делаю новую колонку Турбо, чтобы выделять машины с турбированным двигателем
# а также разделяю пробег от километров и делаю из этого целые числа
df1['Turbo'] = 0
df1['Turbo'] = turbo(df1)
df1['Engine_volume'] = [volume(i) for i in df1['Engine_volume']]
df1['Engine_volume'] = df1['Engine_volume'].astype(float)
df1.Mileage = [int(''.join(filter(str.isdigit, i ))) for i in df1.Mileage]

In [21]:
# Посмотрим теперь на 0 и 20л двигатели
display(df1[df1['Engine_volume'] == 0])
# Объем двигателя Тойота Аква составляет 1.5 л. Мощность двигателей Toyota Aqua от 74 до 91 л.
#
#Автомобиль комплектовался моторами объёмом: 1.6, 2.0, 2.4 л.
display(df1[df1['Engine_volume'] == 20])
# То есть это тоже неккоретные данные,которые мы можем убрать
ind1 = df1[df1['Engine_volume'] == 0].index
df1 = df1.drop(index=ind1)
inde1 = df1[df1['Engine_volume'] == 20].index
df1 = df1.drop(index=inde1)

,Brand,Model,Year_of_production,Type,Leather_interior,Fuel_type,Engine_volume,Mileage,Cylinders,Gear_box,Drive_wheels,Doors,Color,Airbags,Tax,Turbo
1482,TOYOTA,Aqua,2012,Hatchback,Yes,Petrol,0.0,118000,4.0,Automatic,Front,04-May,Grey,4,87,0


,Brand,Model,Year_of_production,Type,Leather_interior,Fuel_type,Engine_volume,Mileage,Cylinders,Gear_box,Drive_wheels,Doors,Color,Airbags,Tax,Turbo
1561,HYUNDAI,Sonata,2014,Sedan,Yes,LPG,20.0,333686,4.0,Automatic,Front,04-May,Silver,4,5603,0


In [26]:
# теперь посмотрим на дубликаты и уберем их, тк есть полностью одинаковые машины, но по цене разной - странно
# цена разная с дубликатами - находил в траин
df1_duplicates = df1.copy()
dupl_rows1 = df1_duplicates[df1_duplicates.duplicated()].index
df1 = df1.drop(index=dupl_rows1)
df1 = df1.drop_duplicates(keep = 'first')

In [35]:
display(df1.Mileage.nlargest(5)) # тут все нормально
display(df1.Mileage.nsmallest(5)) # тут очень много нулей
display(df1[df1.Mileage==0]) # тоже вот очень странно(машинам по 10 и более лет некоторым, а у них 0 пробег)
# я думаю, что это тоже ошибки в данных, поэтому удаляю
ii1 = df1[df1.Mileage==0].index
df1 = df1.drop(index=ii1)

1879    1456394
1135    1440000
761      820000
201      789888
1593     735970
Name: Mileage, dtype: int64

229     1000
2204    1000
175     1166
1815    1275
246     1405
Name: Mileage, dtype: int64

,Brand,Model,Year_of_production,Type,Leather_interior,Fuel_type,Engine_volume,Mileage,Cylinders,Gear_box,Drive_wheels,Doors,Color,Airbags,Tax,Turbo


In [44]:
# посмотрим как отмечал выше на 0 и 20л двигатель
display(df1[df1['Engine_volume'] == 0])
display(df1[df1['Engine_volume'] == 20])
# это тоже является некорректными данными, которые не соответствуют действительности
## Объем двигателя Тойота Аква составляет 1.5 л. Мощность двигателей Toyota Aqua от 74 до 91 л.
##Автомобиль комплектовался моторами объёмом: 1.6, 2.0, 2.4 л.(Соната)
#удаляем
i1 = df1[df1['Engine_volume'] == 0].index
df1 = df1.drop(index=i1)

iii1 = df1[df1['Engine_volume'] == 20].index
df1 = df1.drop(index=iii1)

,Brand,Model,Year_of_production,Type,Leather_interior,Fuel_type,Engine_volume,Mileage,Cylinders,Gear_box,Drive_wheels,Doors,Color,Airbags,Tax,Turbo
1482,TOYOTA,Aqua,2012,Hatchback,Yes,Petrol,0.0,118000,4.0,Automatic,Front,04-May,Grey,4,87,0


,Brand,Model,Year_of_production,Type,Leather_interior,Fuel_type,Engine_volume,Mileage,Cylinders,Gear_box,Drive_wheels,Doors,Color,Airbags,Tax,Turbo
1561,HYUNDAI,Sonata,2014,Sedan,Yes,LPG,20.0,333686,4.0,Automatic,Front,04-May,Silver,4,5603,0


In [46]:
df1.Tax.nsmallest(5)
# тут все нормально, я считаю

1963    155
1864    259
136     271
161     271
1243    271
Name: Tax, dtype: int64

In [48]:
df1.Tax.nlargest(5) 
# тут все нормально, я считаю

1463    2913
1828    2736
326     2628
974     2455
209     2297
Name: Tax, dtype: int64

In [52]:
display(df1['Year_of_production'].nsmallest(5))
display(df1[df1['Year_of_production']==1995])
# данные нормальные, все нормально

413     1995
458     2000
926     2000
2176    2000
381     2001
Name: Year_of_production, dtype: int64

,Brand,Model,Year_of_production,Type,Leather_interior,Fuel_type,Engine_volume,Mileage,Cylinders,Gear_box,Drive_wheels,Doors,Color,Airbags,Tax,Turbo
413,MERCEDES-BENZ,C 180,1995,Sedan,No,Petrol,1.8,2014,4.0,Manual,Rear,04-May,Black,2,1529,0


In [ ]:
# Поскольку в БЛОК 2 после дубликатов я убираю Model, тк не несет доп информации, а я беру Бренд
# то я удаляю эту переменную 

In [53]:
df1 = df1.drop(columns=['Model'])

### Сделаю Сolumn Transformer для теста

In [57]:
cat_features = ['Brand','Type','Color']
num_features = ['Year_of_production','Mileage','Tax']
columns = ['Brand','Type','Color','Fuel_type','Gear_box','Drive_wheels','Year_of_production','Mileage','Tax','Leather_interior','Doors','Engine_volume','Cylinders','Airbags','Turbo']
X = df1.copy()

In [58]:
fuel_type_map = [{
    'col':'Fuel_type', 
    'mapping':{'Diesel':3,'Petrol':2,'Hybrid':1,'LPG':0}
    }]
gear_box_map = [{
    'col':'Gear_box',
    'mapping':{'Tiptronic':3,'Automatic':2,'Variator':1,'Manual':0}
    }]
drive_wheels_map = [{
    'col':'Drive_wheels',
    'mapping':{'4x4':2,'Rear':1,'Front':0}
    }]
doors_map = [{
    'col':'Doors',
    'mapping':{'04-May':2,'>5':1,'02-Mar':0}
    }]
leather_interior_map = [{
    'col':'Leather_interior',
    'mapping':{'Yes':1,'No':0}
    }]

fuel_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = fuel_type_map))])

gear_box_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = gear_box_map))])

drive_wheels_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = drive_wheels_map))])

doors_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = doors_map))])

leather_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = leather_interior_map))])

   
# для деревьев можно не стандартизировать данные в фичах 
CT = ColumnTransformer(transformers=[('cat',OrdinalEncoder(),cat_features),
                                     ('fuel_type', fuel_transformer, ['Fuel_type']),
                                     ('gear_box', gear_box_transformer, ['Gear_box']),
                                     ('drive_wheels', drive_wheels_transformer, ['Drive_wheels']),
                                     ('num','passthrough',num_features), #MinMaxScaler()
                                     ('leather',leather_transformer,['Leather_interior']),
                                     ('doors', doors_transformer, ['Doors'])],
                                      remainder = 'passthrough')

display(CT)

X_CT = CT.fit_transform(X)#.toarray()
X_CT = pd.DataFrame(X_CT,columns=columns)
X_CT.head()

ColumnTransformer(remainder='passthrough',
                  transformers=[('cat', OrdinalEncoder(),
                                 ['Brand', 'Type', 'Color']),
                                ('fuel_type',
                                 Pipeline(steps=[('ce',
                                                  OrdinalEncoder(mapping=[{'col': 'Fuel_type',
                                                                           'mapping': {'Diesel': 3,
                                                                                       'Hybrid': 1,
                                                                                       'LPG': 0,
                                                                                       'Petrol': 2}}]))]),
                                 ['Fuel_type']),
                                ('gear_box',
                                 Pipeline(steps=[('ce',
                                                  OrdinalEncoder(mapping=[{'col': 'Gear_box',
                                                                           'mapping': {'Auto...
                                                                                       'Rear': 1}}]))]),
                                 ['Drive_wheels']),
                                ('num', 'passthrough',
                                 ['Year_of_production', 'Mileage', 'Tax']),
                                ('leather',
                                 Pipeline(steps=[('ce',
                                                  OrdinalEncoder(mapping=[{'col': 'Leather_interior',
                                                                           'mapping': {'No': 0,
                                                                                       'Yes': 1}}]))]),
                                 ['Leather_interior']),
                                ('doors',
                                 Pipeline(steps=[('ce',
                                                  OrdinalEncoder(mapping=[{'col': 'Doors',
                                                                           'mapping': {'02-Mar': 0,
                                                                                       '04-May': 2,
                                                                                       '>5': 1}}]))]),
                                 ['Doors'])])

,Brand,Type,Color,Fuel_type,Gear_box,Drive_wheels,Year_of_production,Mileage,Tax,Leather_interior,Doors,Engine_volume,Cylinders,Airbags,Turbo
0,28.0,3.0,2.0,1.0,1.0,0.0,2009.0,174000.0,878.0,0.0,2.0,1.8,4.0,8.0,0.0
1,10.0,1.0,1.0,2.0,2.0,0.0,2014.0,58000.0,749.0,1.0,2.0,2.4,4.0,4.0,0.0
2,26.0,4.0,14.0,2.0,2.0,2.0,2015.0,90392.0,934.0,1.0,2.0,2.5,4.0,0.0,0.0
3,11.0,8.0,14.0,2.0,2.0,0.0,2018.0,41895.0,1079.0,1.0,2.0,2.0,4.0,4.0,0.0
4,13.0,4.0,11.0,2.0,2.0,0.0,2009.0,163866.0,1141.0,1.0,2.0,2.4,4.0,4.0,0.0


In [61]:
# возьму преобразование Бокса-Кокса для пробег и налога
# преобраазуем переменные пробег и налог
power = PowerTransformer(method='box-cox') 
minmax = MinMaxScaler()

mile = power.fit_transform(minmax.fit_transform(pd.DataFrame(X_CT.Mileage)) + 1)
tax = power.fit_transform(minmax.fit_transform(pd.DataFrame(X_CT.Tax)) + 1)


X_CT.Mileage = mile
X_CT.Tax = tax
#преобразовываю данные и записываю в колонки моего Х после трансформера
# скорее всего это преобразование буду делать до трансформера,то есть сразу преобразовывать фрейм по этим колонкам
# тк трудно будет сделать преобразования после pipeline

In [ ]:
# Трансформер для теста работает, поэтому все нормально
# данные почистили,трансформер проверили
# буду срезать тоже на нем неинформативные переменные, но это уже буду делать в финальном файле